In [1]:
%matplotlib inline

In [3]:
import numpy
import numpy as np
import matplotlib
from matplotlib import pyplot
from matplotlib import pyplot as plt
import os
import pynbody


In [ ]:
#### AREPOSNAP-UTIL packages ####
# in following packages some of the parameters might be useful to change. still have to figure out which

# I think areposnap.gadget is mostly to read in data and do some transformation with them... did not find
from areposnap.gadget import gadget_readsnap
'''
def gadget_readsnap( snapshot, snappath=".", snapbase="snapshot_", verbose=False, onlyHeader=False, \
chunk=False, loadonly=False, loadonlytype=False, nommap=True, tracer=False,hdf5=True, forcesingleprec=False,\
cosmological=None, loadonlyhalo=-1, subfind=None, species=None, lazy_load=False ):
    if (not (os.path.exists( "%s/%s%03d" % (snappath, snapbase, snapshot) ) or \
    os.path.exists( "%s/%s%03d.0" % (snappath, snapbase, snapshot) ))) and \
    (os.path.exists( "%s/snapdir_%03d/%s%03d.0" % (snappath, snapshot, snapbase, snapshot)) \
    or os.path.exists( "%s/snapdir_%03d/%s%03d.0.hdf5" % (snappath, snapshot, snapbase, snapshot)) \
    or os.path.exists( "%s/snapdir_%03d/%s%03d.0.h5" % (snappath, snapshot, snapbase, snapshot)) ):
        snappath += "/snapdir_%03d/" % (snapshot)
    return gadget_snap.gadget_snapshot( "%s/%s%03d" % (snappath, snapbase, snapshot), \
    verbose=verbose, chunk=chunk, loadonly=loadonly, loadonlytype=loadonlytype, nommap=nommap, \
    onlyHeader=onlyHeader, tracer=tracer, hdf5=hdf5, forcesingleprec=forcesingleprec, \
    cosmological=cosmological, loadonlyhalo=loadonlyhalo, subfind=subfind, species=species, lazy_load=lazy_load )
'''

from areposnap.gadget_subfind import load_subfind
'''
def load_subfind( id, base="fof_subhalo_tab_", dir="", verbose=False, loadonly=False, \
hdf5=True, forcesingleprec=False, cosmological=None, onlyHeader=False ):
    return subfind( id, base, dir, verbose, loadonly=loadonly, hdf5=hdf5, \
    forcesingleprec=forcesingleprec, cosmological=cosmological, onlyHeader=onlyHeader )
'''
import areposnap.gadget_snap
# has some nice parameters inside, do not know how to access them yet. that is what this notebook is for
# but is already called by gadget_readsnap... need to figure out how to to access their really nice parameters
# figured it out: just certain particle types have certain parameters ... stars do not have that many

# some other arepo-snap util functions which might be useful to me; not loading from the snapshots but from other files
# loading energies
from areposnap.gadget import gadget_readenergy
'''
def gadget_readenergy(snappath="./"):
    """ convenience function """
    return gadget_energyfile(snappath)
'''

# seems to have ang mom inside
from areposnap.gadget import gadget_binaryfile
'''
def gadget_readbinary(snappath="./"):
    return gadget_binaryfile(snappath)
'''

# could be what we need but I have to figure out what exactly it does and what it needs, especially chunknum and treenum
from areposnap.gadget_tree import load_tree, load_tree_fof
'''
def load_tree(chunknum, treenum ,base="", directory="", verbose=False, loadonly=False, hdf5=True):
    return tree(chunknum, treenum, base, directory, verbose, loadonly=loadonly, hdf5=hdf5 )

def load_tree_fof(id,fof_tree_catalog,base="", directory="", verbose=False, loadonly=False, hdf5=True):

    chunknums,treenums,fofID,subhaloID,treeIDs = genfromtxt(fof_tree_catalog,dtype = int,unpack=True,skip_header=2)
    index = where(fofID == id)
    if verbose:
        print( 'number of trees: ',len(index[0]) )
    if len(index[0]) == 0:
        print( 'no trees found!' )
        return None
    chunknum = chunknums[index[0][0]]
    treenum = treenums[index[0][0]]
    treeID = treeIDs[index[0][0]]

    return tree(chunknum, treenum, base, directory, verbose, loadonly=loadonly, hdf5=hdf5 ),treeID
'''

In [ ]:
# written by Timo Halbesma (MPA); added some additional comments for better understanding
def eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[0,1,2,3,4],
                     haloid=0, galradfac=0.1, verbose=True):
    """ Method to eat an Auriga snapshot, given a level/halo_number/snapnr.
        Subfind has been executed 'on-the-fly', during the simulation run.

        @param level: level of the Auriga simulation (3=high, 4='normal' or 5=low).
            Level 3/5 only for halo 6, 16 and 24. See Grand+ 2017 for details.
            Careful when level != 4 because directories may have different names.
        @param halo_number: which Auriga galaxy? See Grand+ 2017 for details.
            Should be an integer in range(1, 31)
        @param snapnr: which snapshot number? This is an integer, in most cases
            in range(1, 128) depending on the number of timesteps of the run.
            The last snapshot would then be 127. Snapshots are written at a
            certain time, but careful because the variable called time is actually
            the cosmological expansion factor a = 1/(1+z). For example, snapnr=127
            has s.time = 1, which corresponds to a redshift of ~0. This makes sense
            because this is the last snapshot and the last snapshot is written at
            redshift zero
        @param snappath: full path to the level/halo directory that contains
            all of the simulation snapshots
        @param loadonlytype: which particle types should be loaded? This should
            be a list of integers. If I'm not mistaken, the options are:
            0 (gas), 1 (halo), 2 (disk), 3 (bulge), 4 (stars), 5 (black holes).
            So to get the dark matter: load particles 1/2/3. In zoom-simulations
            particletype 3 may be used for low-resolution particles in the outer
            regions and they should not be present in (and contaminating) the
            inner region. I'm not too sure of the latter though.
        @param haloid: the ID of the SubFind halo. In case you are interested
            in the main galaxy in the simulation run: set haloid to zero.
            This was a bit confusing to me at first because a zoom-simulation run
            of one Auriga galaxy is also referred to as 'halo', see halo_number.
        @param galradfac: the radius of the galaxy is often used to make cuts in
            the (star) particles. It seems that in general galrad is set to 10%
            of the virial radius R200 of the DM halo that the galaxy sits in. The
            disk does seem to 'end' at 0.1R200.
        @param verbose: boolean to print some information

        @return: two-tuple (s, sf) where s is an instance of the gadget_snapshot
            class, and sf is an instance of the subfind class. See Arepo-snap-util,
            gadget_snap.py respectively gadget_subfind.py """

    # Eat the subfind friend of friends output
    sf = load_subfind(snapnr, dir=snappath)

    # Eat the Gadget snapshot
    s = gadget_readsnap(snapnr, snappath=snappath, lazy_load=True,
        subfind=sf, loadonlytype=loadonlytype)
    s.subfind = sf

    # Sets s.(sub)halo. This allows selecting the halo, e.g. 0 (main 'Galaxy')
    s.calc_sf_indizes(s.subfind)
    # Note that selecting the halo now rotates the disk using the principal axis.
    # rotate_disk is a general switch which has to be set to True to rotate.
    # To then actually do the rotation, do_rotation has to be True as well.
    # Within rotate_disk there are three methods to handle the rotation. Choose
    # one of them, but see the select_halo method for details.
    s.select_halo( s.subfind, haloid=haloid, galradfac=galradfac,
        rotate_disk=True, use_principal_axis=True, euler_rotation=False,
        use_cold_gas_spin=False, do_rotation=True )

    # Sneak some more info into the s instance
    s.halo_number = halo_number
    s.level = level
    s.snapnr = snapnr
    s.haloid = haloid

    # This means that galrad is 10 % of R200 (200*rho_crit definition)
    s.galrad = galradfac * sf.data['frc2'][haloid]  # frc2 = Group_R_Crit200

    if verbose:
        print("\ngalrad  : {0}".format(s.galrad))
        print("redshift: {0}".format(s.redshift))
        print("time    : {0}".format(s.time))
        print("center  : {0}\n".format(s.center))

    return s, sf


In [ ]:
def get_cartesian_vectors(s, sf, mask):
    x,  y,  z  = s.pos[::,2][mask], s.pos[::,1][mask], s.pos[::,0][mask]
    vx, vy, vz = s.vel[::,2][mask], s.vel[::,1][mask], s.vel[::,0][mask]

    # All three expressions for rxyz compute the same thing :-).
    # rxyz = numpy.sqrt( x**2 + y**2 + z**2)
    # rxyz = numpy.sqrt( (s.pos[mask]**2).sum(axis=1) )
    rxyz = s.r()[mask]

    # Both expressions for rxy compute the same thing :-).
    # rxy = numpy.sqrt( (s.pos[mask][:,1:]**2).sum(axis=1) )
    rxy = numpy.sqrt( x**2 + y**2 )

    return (x, y, z), (vx, vy, vz), rxyz, rxy


In [ ]:
level = 4
basedir = "/Users/smilanov/Desktop/Auriga/level{0}/".format(level)
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"
    for snapnr in range(127, 128, 1):
        print("level   : {0}".format(level))
        print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print("basedir : {0}".format(basedir))
        print("halodir : {0}".format(halodir))
        print("snappath: {0}\n".format(snappath))

        s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath,
            loadonlytype=[1,2,3,4], haloid=0, galradfac=0.1, verbose=True)

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = numpy.maximum( s.data['gmet'], 1e-40 )

In [ ]:
com = s.getCenterOfMass()
angc = s.getAngularMomentumCore()

In [ ]:
'''
s.id should be super useful following a particle through the different snapshots
buuut: 
    find the fastest way to compare a list of subsets to the main Id list (containing more than 4million IDs)
    it can happen that some particles are created along the way and not there from the beginning
'''
s.id

In [ ]:
# I think this is the number of subhalos of the 0th halo.
Nsubhalos = sf.data['fnsh'][0] # = 917
print('Number of subhalos:', Nsubhalos)

# this is the number of stars of each subhalo... as I see most of them are just DM (maybe gas) subhalos
subhalosLenStars = sf.data['slty'][0:Nsubhalos,4]
#print(subhalosLenStars)

# select subhalos containing stars
isubs, = np.where(subhalosLenStars > 0)
print(isubs)

In [ ]:
# so I need to mark particles of the dwarf galaxies and get their ID 
# (in order to get position and velocity at each timestep)



In [ ]:
s.calc_sf_indizes( sf)

In [ ]:
s.subhalo == 1

In [ ]:
istars, = numpy.where( (s.type == 4) & (s.halo == 0))
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, istars)
#xt, yt, zt = x[im], y[im], z[im]
fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(12, 6))

ax1.hist2d(1000*x, 1000*y, bins=401, norm=matplotlib.colors.LogNorm())
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_aspect('equal')
ax2.hist2d(1000*rxy, 1000*z, bins=401, norm=matplotlib.colors.LogNorm())
ax2.set_xlabel("R")
ax2.set_ylabel("z")
ax2.set_aspect('equal')
fig.show()

'''istars, = numpy.where( (s.type == 4) & (s.halo == 0) & (s.subhalo == 1))
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, istars)
#xt, yt, zt = x[im], y[im], z[im]
fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(12, 6))

ax1.hist2d(1000*x, 1000*y, bins=401, norm=matplotlib.colors.LogNorm())
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_aspect('equal')
ax2.hist2d(1000*rxy, 1000*z, bins=401, norm=matplotlib.colors.LogNorm())
ax2.set_xlabel("R")
ax2.set_ylabel("z")
ax2.set_aspect('equal')
print(istars)
print(s.id[istars])'''

In [ ]:
ids, ty, x, y, Z, vx, vy, vz, m = np.loadtxt('../data/level_5_halo_24_snapshot_63_data.txt',  \
                                             dtype=[('f0', '<i8'), ('f1', '<i8'), ('f2', '<f8'), \
                                                    ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8'), \
                                                    ('f6', '<f8'), ('f7', '<f8'), ('f8', '<f8')], unpack = True)

In [ ]:
Rxy = np.sqrt(x**2 + y**2)
rxyz = np.sqrt(x**2 + y**2 + z**2)

In [ ]:
def dens_3d(r0, r1, mass):
    if r1 > r0:
        rdiff = r1**3 - r0**3
    else:
        rdiff = r0**3 - r1**3        
    vol = 4./3. * np.pi * rdiff
    return vol

def binning(r_vec, num = 50, scale = 'lin'):
    r_min = np.min(r_vec)
    r_max = np.max(r_vec)
    if scale == 'lin':
        r = np.linspace(r_min, r_max, num = num)
    elif scale == 'log':
        r = np.geomspace(r_min, r_max, num = num)
    else:
        sys.exit('Binning scale not yet defined.')
    return r


In [ ]:
r_bins = binning(Rxy)
bin_ind = np.digitize(Rxy, r_bins)
z_bins = binning(Z)
zbin_ind = np.digitize(Z, z_bins)
mass_arr_r = []
mass_arr_z = []
for j in range (50):
    mass_arr_r.append(np.sum(m[bin_ind == j+1]))
for k in range (50):
    mass_arr_z.append(np.sum(m[zbin_ind == k+1]))
fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(12, 6))
ax1.plot(1000*r_bins, mass_arr_r, 'k.')

ax1.set_xlabel("R")
ax1.set_ylabel("mass")
#ax1.set_aspect('equal')
ax2.hist2d(1000*rxy, 1000*z, bins=401, norm=matplotlib.colors.LogNorm())
ax2.set_xlabel("R")
ax2.set_ylabel("z")
ax2.set_aspect('equal')
fig.show()
fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(12, 6))
ax1.plot(1000*z_bins, mass_arr_z, 'k.')

ax1.set_xlabel("z")
ax1.set_ylabel("mass")
#ax1.set_aspect('equal')
ax2.hist2d(1000*rxy, 1000*z, bins=401, norm=matplotlib.colors.LogNorm())
ax2.set_xlabel("R")
ax2.set_ylabel("z")
ax2.set_aspect('equal')
fig.show()

In [ ]:
istars, = numpy.where( (s.type == 4) & (s.halo == 0) & (s.r() > 10*s.galrad) )
# check if exclusion of central galaxy is necessary
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, istars)

import sklearn.cluster as cl
# n_cluster = 7 was better
X = numpy.vstack((x,y,z)).T
kmeans = cl.KMeans(n_clusters=9).fit(X)
centers = kmeans.cluster_centers_

In [ ]:
print(np.shape(centers))

In [ ]:
fig, (ax1, ax2) = pyplot.subplots(1, 2, figsize=(12, 6))

ax1.plot(x, y, "kD", ms=1, alpha=0.1, c = 'yellow')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.scatter(centers[ :, 0], centers[:,1], c = 'red', zorder = 5)
ax2.plot(x, z, "kD", ms=1, alpha=0.1)
ax2.scatter(centers[:,0],centers[:,2], c='red', zorder = 5)
ax2.set_xlabel("R")
ax2.set_ylabel("z")

In [ ]:
hist, xedges, yedges = numpy.histogram2d(1000.*x[istars], 1000.*y[istars], bins = 100)
#maxval = numpy.max(hist)
maxindex = (-hist).argsort()
maxhist = (-hist).sort()

for i, item in enumerate(maxhist):
    ind = hist.index(item)
    targetval = 0.1*item
    outinds = np.where(hist < targetval)
    #foundinds = bisect()
    
    if i == 20: break
    

In [ ]:
ga

In [ ]:
gadget_snap.plot_radmasspro

In [ ]:
from areposnap.gadget_snap import *

In [ ]:
snapshot = snapnr
snapbase="snapdir_"
snapbase2="snapshot_"
if (not (os.path.exists( "%s/%s%03d" % (snappath, snapbase, snapshot) ) or os.path.exists( "%s/%s%03d.0" % (snappath, snapbase, snapshot) ))) and (os.path.exists( "%s/snapdir_%03d/%s%03d.0" % (snappath, snapshot, snapbase, snapshot)) or os.path.exists( "%s/snapdir_%03d/%s%03d.0.hdf5" % (snappath, snapshot, snapbase, snapshot)) or os.path.exists( "%s/snapdir_%03d/%s%03d.0.h5" % (snappath, snapshot, snapbase, snapshot)) ):
    snappath += "/snapdir_%03d/" % (snapshot)
testshot = gadget_snapshot("%s%s%03d/%s%03d" % (snappath, snapbase, snapnr, snapbase2, snapnr))

In [ ]:
testshot.plot_radmassprof()

In [ ]:
testshot.

In [4]:
pynbody.load?